In [2]:
from dotenv import load_dotenv 
load_dotenv()

True

In [38]:
from langchain.chat_models import init_chat_model
from pydantic import BaseModel, Field
from typing_extensions import TypedDict, Annotated
from langchain.agents import create_agent
from dataclasses import dataclass

In [4]:
model = init_chat_model("groq:qwen/qwen3-32b")

In [6]:
class Movie(BaseModel):
    title: str = Field(description = "The title of the movie")
    year: int = Field(description = "This year the movie was released")
    director: str = Field(description = "The director of the movie")
    rating: float = Field(description = "The movie's rating out of 10")

In [7]:
model_with_structured_op = model.with_structured_output(Movie)
model_with_structured_op

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001F7A1808A50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001F7A193A6D0>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Movie', 'description': '', 'parameters': {'properties': {'title': {'description': 'The title of the movie', 'type': 'string'}, 'year': {'description': 'This year the movie was released', 'type': 'integer'}, 'director': {'description': 'The director of the movie', 'type': 'string'}, 'rating': {'description': "The movie's rating out of 10", 'type': 'number'}}, 'required': ['title', 'year', 

In [8]:
model.invoke("Provide details about the movie Inception.")

AIMessage(content='<think>\nOkay, so I need to provide details about the movie Inception. Let me start by recalling what I know about it. Inception is a science fiction movie directed by Christopher Nolan. It stars Leonardo DiCaprio, right? The main character is Dom Cobb, a professional thief who steals information by infiltrating the subconscious of his targets. But in this movie, instead of stealing secrets, he wants to plant an idea into someone\'s mind. That concept is called "inception." \n\nI think the movie uses a lot of layered realities. Cobb and his team use a machine called the "Dreamshare" to enter people\'s dreams. They have different levels of dreams, like level one, two, three, and even limbo. Each level is deeper and more unstable. The deeper they go, the more time passes in the real world. Time dilation is a key element here. For example, one hour in the first dream level equals five minutes in the real world, but in the deeper levels, it\'s even more exaggerated.\n\nT

In [10]:
response = model_with_structured_op.invoke("Provide details about the movie Inception.")
response

Movie(title='Inception', year=2010, director='Christopher Nolan', rating=8.8)

In [11]:
class Movie(BaseModel):
    """A movie with details."""
    title: str = Field(..., description = "The title of the movie")
    year: str = Field(..., description = "The year the movie was released")
    director: str = Field(..., description = "The director of the movie")
    rating: float = Field(..., description = "The movie's rating out of 10")

In [14]:
model_with_structured_op = model.with_structured_output(Movie, include_raw = True)

In [15]:
response = model_with_structured_op.invoke("Provide details about the movie Inception")
response

{'raw': AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for details about the movie Inception. Let me see what I need to do. The available tool is the Movie function, which requires title, year, director, and rating. I need to provide those parameters. \n\nFirst, I know that Inception is a 2010 movie directed by Christopher Nolan. The title is "Inception", so that\'s straightforward. The year would be 2010. The director is Christopher Nolan. For the rating, I think it\'s around 8.8 on IMDb, but maybe I should check to be sure. But since I can\'t actually look it up, I\'ll go with what I remember.\n\nWait, the user just wants the details, so I should structure the function call with those parameters. Let me confirm again: title is "Inception", year 2010, director Christopher Nolan, and rating 8.8. That should cover all required fields. I think that\'s correct. Let me put that into the tool call format.\n', 'tool_calls': [{'id': '0t37g3wy1', 'funct

In [16]:
class Actor(BaseModel):
    name: str
    role: str

In [17]:
class MovieDetails(BaseModel):
    title: str
    year: int
    cast: list[Actor]
    genres: list[str]
    budget: float | None = Field(None, description = "Budget in millions USD")

In [18]:
model_with_structured_op = model.with_structured_output(MovieDetails)

In [19]:
response = model_with_structured_op.invoke("Provide details about the movie Inception.")
response

MovieDetails(title='Inception', year=2010, cast=[Actor(name='Leonardo DiCaprio', role='Dom Cobb'), Actor(name='Joseph Gordon-Levitt', role='Arthur'), Actor(name='Ellen Page', role='Ariadne'), Actor(name='Tom Hardy', role='Eames')], genres=['Science Fiction', 'Action', 'Thriller'], budget=None)

In [21]:
class MovieDict(TypedDict):
    """A movie with details."""
    title: Annotated[str, ..., "The title of the movie"]
    year: Annotated[int, ..., "The year the movie was released"]
    director: Annotated[str, ..., "The director of the movie"]
    rating: Annotated[float, ..., "The movie's rating out of 10"]

In [22]:
model_with_td = model.with_structured_output(MovieDict)

In [23]:
response = model_with_td.invoke("Please provide the detials of the movie avangers")
response

{'director': 'Joss Whedon', 'rating': 8, 'title': 'Avengers', 'year': 2012}

In [24]:
class Actor(TypedDict):
    name: str
    role: str

In [25]:
class MovieDetails(TypedDict):
    title: str
    year: str
    cast: list[Actor]
    genres: list[str]
    budget: float | None = Field(None, description = "Budget in millions USD")

In [26]:
model_with_td_op = model.with_structured_output(MovieDetails)

In [27]:
response = model_with_td_op.invoke("Provide details about the movie Inception")
response

{'budget': 160000000,
 'cast': [{'name': 'Leonardo DiCaprio', 'role': 'Dom Cobb'},
  {'name': 'Joseph Gordon-Levitt', 'role': 'Arthur'},
  {'name': 'Ellen Page', 'role': 'Ariadne'},
  {'name': 'Tom Hardy', 'role': 'Eames'}],
 'genres': ['Science Fiction', 'Action', 'Thriller'],
 'title': 'Inception',
 'year': '2010'}

In [28]:
model.profile

{'max_input_tokens': 131072,
 'max_output_tokens': 16384,
 'image_inputs': False,
 'audio_inputs': False,
 'video_inputs': False,
 'image_outputs': False,
 'audio_outputs': False,
 'video_outputs': False,
 'reasoning_output': True,
 'tool_calling': True}

In [30]:
class ContactInfo(BaseModel):
    """Contact information for a person."""
    name: str = Field(description = "The name of the person")
    email: str = Field(description = "The email address of the person")
    phone: str = Field(description = "The phone number of the person")

In [31]:
agent = create_agent(
    model = model,
    response_format = ContactInfo
)

In [32]:
result = agent.invoke({
    "messages": [{"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"}]
})
print(result["structured_response"])

name='John Doe' email='john@example.com' phone='(555) 123-4567'


In [33]:
class ContactInfo(TypedDict):
    """Contact information for a person."""
    name: str 
    email: str 
    phone: str 

In [36]:
agent = create_agent(
    model = model,
    response_format = ContactInfo
)

In [37]:
result = agent.invoke({
    "messages": [{"role": "user", "content": "Extract conact info from: John Doe, john@example.com, (555) 123-4567"}]
})
result["structured_response"]

{'name': 'John Doe', 'email': 'john@example.com', 'phone': '(555) 123-4567'}

In [39]:
@dataclass
class ContactInfo:
    """Contact information for a person.""" 
    name: str 
    email: str 
    phone: str

In [40]:
agent = create_agent(
    model = model,
    response_format = ContactInfo
)

In [42]:
result = agent.invoke({
    "messages": [{"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-456"}]
})
result["structured_response"]

ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-456')